### 

In [27]:
import os

import numpy as np
from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras.optimizers import Adam
import dataset

In [28]:
(train_data, _), (test_data, y_test) = dataset.get_normalized_data()
print(train_data.shape)  # 60000, 784 for training the algorithm
print(test_data.shape)  # 10000, 784 for evaluating performance on unseen data
(_, IMG_SIZE) = train_data.shape

(60000, 784)
(10000, 784)


In [21]:
LOG_DIR = "logs"
BATCH_SIZE = 256
EPOCHS = 10
ENCODING_DIM = 3

In [22]:
input_img = Input(shape=(IMG_SIZE,))
encoded = Dense(512, activation='elu')(input_img)
encoded = Dense(128, activation='elu')(encoded)
encoded = Dense(ENCODING_DIM, name="features")(encoded)
encoded_activated = Activation('linear')(encoded)
# Fit in another input layer to be able to extract decoder
decoded = Dense(128, activation='elu')(encoded_activated)
decoded = Dense(512, activation='elu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(), loss='mse')
encoder = Model(input_img, encoded)

encoded_input = Input(shape=(ENCODING_DIM,))
decoder = autoencoder.layers[-3](encoded_input)
decoder = autoencoder.layers[-2](decoder)
decoder = autoencoder.layers[-1](decoder)

# create the decoder model
decoder = Model(encoded_input, decoder)

In [23]:
# save class labels to disk to color data points in TensorBoard accordingly

os.makedirs(LOG_DIR, exist_ok=True)
METADATA_FILE = os.path.join(LOG_DIR, 'metadata.tsv')
with open(METADATA_FILE, 'w') as f:
    np.savetxt(f, y_test)
tensorboard = TensorBoard(batch_size=BATCH_SIZE,
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=test_data,
                          log_dir=LOG_DIR)

In [24]:
autoencoder.fit(train_data, train_data,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=[tensorboard],
                shuffle=True,
                validation_data=(test_data, test_data))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 58us/step - loss: 0.0593 - val_loss: 0.0460
Epoch 2/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0429 - val_loss: 0.0404
Epoch 3/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.0392 - val_loss: 0.0380
Epoch 4/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.0373 - val_loss: 0.0364
Epoch 5/10
60000/60000 [==============================] - 3s 58us/step - loss: 0.0361 - val_loss: 0.0356
Epoch 6/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0352 - val_loss: 0.0349
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch 7/10
60000/60000 [==============================] - 3s 54us/step - loss: 0.0345 - val_loss: 0.0343
Epoch 8/10
60000/60000 [==============================] - 3s 52us/step - loss: 0.0339 - val_loss: 0.0337
Epoch 9/10
60000/60000 [==

In [25]:
os.makedirs('models', exist_ok=True)
autoencoder.save('models/autoencoder.h5')
encoder.save('models/encoder.h5')
decoder.save('models/decoder.h5')